### Imports

In [2]:
import pyspark
pyspark.__version__

'3.5.1'

In [3]:
from pyspark.sql import SparkSession

In [4]:
# Create SparkSession object
spark = SparkSession.builder.master("local[1]").appName('test').getOrCreate()

24/08/10 10:34:27 WARN Utils: Your hostname, codespaces-b3fb74 resolves to a loopback address: 127.0.0.1; using 10.0.2.41 instead (on interface eth0)
24/08/10 10:34:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/10 10:34:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/10 10:34:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# Terminate the cluster
# spark.stop()

In [5]:
spark.version

'3.5.1'

### Read a csv file

In [5]:
# Read data from CSV file
df = spark.read.csv('flights.csv',
                         sep=',',
                         header=True,
                         inferSchema=True,
                         nullValue='NA')

# Get number of records
print("The data contain %d records." % df.count())

# View the first five records
display(df.show(5))

# Check column data types
print(df.dtypes)

The data contain 15001 records.
+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 10| 10|  1|     OO|  5836|ORD| 157|  8.18|      51|   27|
|  1|  4|  1|     OO|  5866|ORD| 466|  15.5|     102| NULL|
| 11| 22|  1|     OO|  6016|ORD| 738|  7.17|     127|  -19|
|  2| 14|  5|     B6|   199|JFK|2248| 21.17|     365|   60|
|  5| 25|  3|     WN|  1675|SJC| 386| 12.92|      85|   22|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



None

[('mon', 'int'), ('dom', 'int'), ('dow', 'int'), ('carrier', 'string'), ('flight', 'int'), ('org', 'string'), ('mile', 'int'), ('depart', 'double'), ('duration', 'int'), ('delay', 'int')]


In [30]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
# from pyspark.sql.types import *

# Specify column names and types
schema = StructType([
    StructField("mon", FloatType()),
    StructField("dom", IntegerType()),
    StructField("dow", IntegerType()),
    StructField("carrier", StringType()),
    StructField("depart", IntegerType()),
    StructField("flight", StringType())
])

# Load data from a delimited file
df = spark.read.csv('flights.csv', sep=',', header=True, schema=schema)

# Print schema of DataFrame
display(df.printSchema())

# View the first five records
display(df.show(5))

root
 |-- mon: float (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- depart: integer (nullable = true)
 |-- flight: string (nullable = true)



None

+----+---+---+-------+------+------+
| mon|dom|dow|carrier|depart|flight|
+----+---+---+-------+------+------+
|10.0| 10|  1|     OO|  5836|   ORD|
| 1.0|  4|  1|     OO|  5866|   ORD|
|11.0| 22|  1|     OO|  6016|   ORD|
| 2.0| 14|  5|     B6|   199|   JFK|
| 5.0| 25|  3|     WN|  1675|   SJC|
+----+---+---+-------+------+------+
only showing top 5 rows



24/08/06 06:20:56 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 10, schema size: 6
CSV file: file:///workspaces/data_science_materials/pyspark/flights.csv


None

### Drop Column & Remove Nulls

In [6]:
# Remove the 'flight' column
df_drop_column = df.drop('flight')
print(df_drop_column.count())

# Number of records with missing 'delay' values
print(df_drop_column.filter('delay IS NULL').count())

# Remove records with missing 'delay' values
df_valid_delay = df_drop_column.filter('delay IS NOT NULL')
print(df_valid_delay.count())

# Remove records with missing values in any column and get the number of remaining rows
df_none_missing = df_valid_delay.dropna()
print(df_none_missing.count())

15001
915
14086
14086


In [7]:
# Import the required function
from pyspark.sql.functions import round

# Convert 'mile' to 'km' and drop 'mile' column (1 mile is equivalent to 1.60934 km)
df_index = df_none_missing.withColumn('km', round(df.mile * 1.60934, 0)) \
                    .drop('mile')

# Create 'label' column indicating whether flight delayed (1) or not (0)
df_index = df_index.withColumn('label', (df_index.delay >= 15).cast('integer'))

# Check first five records
df_index.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|    1|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|    0|
|  2| 14|  5|     B6|JFK| 21.17|     365|   60|3618.0|    1|
|  5| 25|  3|     WN|SJC| 12.92|      85|   22| 621.0|    1|
|  3| 28|  1|     B6|LGA| 13.33|     182|   70|1732.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows



24/08/10 10:34:38 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Categorical to Numerical Encoding

In [8]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(df_index)

# Indexer creates a new column with numeric index values
df_indexed = indexer_model.transform(df_index)

# Repeat the process for the other categorical feature
df_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(df_indexed).transform(df_indexed)
df_indexed.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|carrier_idx|org_idx|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27| 253.0|    1|        2.0|    0.0|
| 11| 22|  1|     OO|ORD|  7.17|     127|  -19|1188.0|    0|        2.0|    0.0|
|  2| 14|  5|     B6|JFK| 21.17|     365|   60|3618.0|    1|        4.0|    2.0|
|  5| 25|  3|     WN|SJC| 12.92|      85|   22| 621.0|    1|        3.0|    4.0|
|  3| 28|  1|     B6|LGA| 13.33|     182|   70|1732.0|    1|        4.0|    3.0|
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+
only showing top 5 rows



### Extracting train and test features

In [17]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'mon', 'dom', 'dow', 'carrier_idx', 'org_idx', 'km', 'depart', 'duration', 'delay'
], outputCol='features')

# Consolidate predictor columns
df_assembled = assembler.transform(df_indexed)

# Check the resulting column
# df_assembled = df_assembled.select('features', 'delay')
df_assembled.select('features', 'delay').show(5, truncate=False)

+-----------------------------------------------+-----+
|features                                       |delay|
+-----------------------------------------------+-----+
|[10.0,10.0,1.0,2.0,0.0,253.0,8.18,51.0,27.0]   |27   |
|[11.0,22.0,1.0,2.0,0.0,1188.0,7.17,127.0,-19.0]|-19  |
|[2.0,14.0,5.0,4.0,2.0,3618.0,21.17,365.0,60.0] |60   |
|[5.0,25.0,3.0,3.0,4.0,621.0,12.92,85.0,22.0]   |22   |
|[3.0,28.0,1.0,4.0,3.0,1732.0,13.33,182.0,70.0] |70   |
+-----------------------------------------------+-----+
only showing top 5 rows



### Train-test Split

In [27]:
# Split into training and testing sets in a 80:20 ratio
df_train, df_test = df_assembled.randomSplit([0.7, 0.3], seed=43)
print(df_train.count(), df_test.count())

# Check that training set has around 80% of records
training_ratio = df_train.count() / df_assembled.count()
print(training_ratio)
df_train.show(5, truncate=False)

9920 4166
0.7042453499929008
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+---------------------------------------------+
|mon|dom|dow|carrier|org|depart|duration|delay|km    |label|carrier_idx|org_idx|features                                     |
+---+---+---+-------+---+------+--------+-----+------+-----+-----------+-------+---------------------------------------------+
|0  |1  |2  |AA     |ORD|8.42  |155     |83   |1617.0|1    |1.0        |0.0    |[0.0,1.0,2.0,1.0,0.0,1617.0,8.42,155.0,83.0] |
|0  |1  |2  |AA     |ORD|15.25 |115     |20   |941.0 |1    |1.0        |0.0    |[0.0,1.0,2.0,1.0,0.0,941.0,15.25,115.0,20.0] |
|0  |1  |2  |AA     |ORD|15.5  |90      |25   |649.0 |1    |1.0        |0.0    |[0.0,1.0,2.0,1.0,0.0,649.0,15.5,90.0,25.0]   |
|0  |1  |2  |AA     |ORD|16.0  |135     |54   |1395.0|1    |1.0        |0.0    |[0.0,1.0,2.0,1.0,0.0,1395.0,16.0,135.0,54.0] |
|0  |1  |2  |AA     |SJC|15.58 |205     |-3   |2314.0|0    |1.0        |4.0    |[0

In [28]:
df_assembled.select('label').filter(df_assembled.label==0).count()

6924

## Decision Tree

### Model Fit

In [29]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(df_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(df_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+-----------+
|label|prediction|probability|
+-----+----------+-----------+
|0    |0.0       |[1.0,0.0]  |
|1    |1.0       |[0.0,1.0]  |
|1    |1.0       |[0.0,1.0]  |
|1    |1.0       |[0.0,1.0]  |
|1    |1.0       |[0.0,1.0]  |
+-----+----------+-----------+
only showing top 5 rows



### Confusion matrix

In [30]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TP + TN)/ (TP + TN + FP + FN)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   19|
|    0|       0.0| 2048|
|    1|       1.0| 2095|
|    0|       1.0|    4|
+-----+----------+-----+

0.9944791166586654


## Logistic Regression

### Model fit & test

In [31]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(df_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(df_test)

# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

24/08/10 10:43:17 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/10 10:43:17 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0| 2052|
|    1|       1.0| 2114|
+-----+----------+-----+



In [32]:
# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: "areaUnderROC"})

precision = 1.00
recall    = 1.00


## Turning Text into Tables

### Text cleaning & vectorization

In [34]:
sms = spark.read.csv('spam.csv', header=True).select(['v2', 'v1']).withColumnRenamed('v2', 'text').dropna()
sms.show(3, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|text                                                                                                                                                       |v1  |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                            |ham |
|Ok lar... Joking wif u oni...                                                                                                                              |ham |
|Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's|spam|
+---------------------

In [35]:
# Import the necessary functions
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer
from pyspark.sql import functions as F

# Remove punctuation (REGEX provided) and numbers
wrangled = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
display(wrangled.show(3, truncate=False))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))
wrangled = wrangled.withColumn('v1', regexp_replace(wrangled.v1, '"""', ''))
wrangled = wrangled.withColumn("label", F.when(wrangled.v1 == "spam", 0).otherwise(1))
display(wrangled.show(3, truncate=False))

# Merge multiple spaces
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))
display(wrangled.show(3, truncate=False))

# Split the text into words
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)

wrangled.show(4, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|text                                                                                                                                                       |v1  |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat                                               |ham |
|Ok lar    Joking wif u oni                                                                                                                                 |ham |
|Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005  Text FA to 87121 to receive entry question std txt rate T&C's apply 08452810075over18's|spam|
+---------------------

None

+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----+
|text                                                                                                                                                       |v1  |label|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----+
|Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat                                               |ham |1    |
|Ok lar    Joking wif u oni                                                                                                                                 |ham |1    |
|Free entry in   a wkly comp to win FA Cup final tkts   st May       Text FA to       to receive entry question std txt rate T&C's apply            over  '

None

+-------------------------------------------------------------------------------------------------------------------------------+----+-----+
|text                                                                                                                           |v1  |label|
+-------------------------------------------------------------------------------------------------------------------------------+----+-----+
|Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat                         |ham |1    |
|Ok lar Joking wif u oni                                                                                                        |ham |1    |
|Free entry in a wkly comp to win FA Cup final tkts st May Text FA to to receive entry question std txt rate T&C's apply over 's|spam|0    |
+-------------------------------------------------------------------------------------------------------------------------------+----+-----+
only showing 

None

+-------------------------------------------------------------------------------------------------------------------------------+----+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                           |v1  |label|words                                                                                                                                                       |
+-------------------------------------------------------------------------------------------------------------------------------+----+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat   

In [36]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

# Remove stop words.
wrangled1 = StopWordsRemover(inputCol='words', outputCol='terms')\
      .transform(wrangled)
display(wrangled1.show(3, truncate=False))

# Apply the hashing trick
wrangled2 = HashingTF(inputCol='terms', outputCol='hash', numFeatures=24)\
      .transform(wrangled1)
display(wrangled2.show(3, truncate=False))

# Convert hashed symbols to TF-IDF
tf_idf = IDF(inputCol='hash', outputCol='features')\
      .fit(wrangled2).transform(wrangled2)
# tf_idf = tf_idf.withColumnRenamed('v1','label')
display(tf_idf.show(3, truncate=False))

+-------------------------------------------------------------------------------------------------------------------------------+----+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                           |v1  |label|words                                                                                                                                                       |terms                                                                                                                              |
+-------------------------------------------------------------------------------------------------------------------------------+----+-----+

None

+-------------------------------------------------------------------------------------------------------------------------------+----+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+
|text                                                                                                                           |v1  |label|words                                                                                                                                                       |terms                                                                                                                              |hash                                                  

None

+-------------------------------------------------------------------------------------------------------------------------------+----+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                           |v1  |label|words                                                                        

None

In [37]:
# Split the data into training and testing sets
sms_train, sms_test = tf_idf.randomSplit([0.4, 0.1], seed=13)

# Fit a Logistic Regression model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# Make predictions on the testing data
prediction = logistic.transform(sms_test)

# Create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   14|
|    0|       0.0|   23|
|    1|       1.0|  929|
|    0|       1.0|  130|
+-----+----------+-----+



## Regression

### One-hot encoding

In [38]:
# Import the one hot encoder class
from pyspark.ml.feature import OneHotEncoder

# Create an instance of the one hot encoder
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights data
onehot = onehot.fit(df_assembled)
df_onehot = onehot.transform(df_assembled)

# Check the results
df_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

+---+-------+-------------+
|org|org_idx|    org_dummy|
+---+-------+-------------+
|ORD|    0.0|(7,[0],[1.0])|
|SFO|    1.0|(7,[1],[1.0])|
|JFK|    2.0|(7,[2],[1.0])|
|LGA|    3.0|(7,[3],[1.0])|
|SJC|    4.0|(7,[4],[1.0])|
|SMF|    5.0|(7,[5],[1.0])|
|TUS|    6.0|(7,[6],[1.0])|
|OGG|    7.0|    (7,[],[])|
+---+-------+-------------+



### Regression model w/o one-hot encoding

In [39]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(df_train)

# Create predictions for the testing data and take a look at the predictions
predictions = regression.transform(df_test)
predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
RegressionEvaluator(labelCol='duration').evaluate(predictions)

24/08/10 10:44:16 WARN Instrumentation: [8602d0ad] regParam is zero, which might cause numerical instability and overfitting.
24/08/10 10:44:17 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+--------+------------------+
|duration|prediction        |
+--------+------------------+
|195     |194.9999999999996 |
|120     |120.00000000000004|
|265     |264.99999999999994|
|65      |65.00000000000027 |
|88      |87.99999999999986 |
+--------+------------------+
only showing top 5 rows



3.1285340969800204e-13

In [40]:
# Intercept (average minutes on ground)
inter = regression.intercept
print(inter)

# Coefficients
coefs = regression.coefficients
print(coefs)

# Average minutes per km
minutes_per_km = regression.coefficients[0]
print(minutes_per_km)

# Average speed in km per hour
avg_speed = 60 / minutes_per_km
print(avg_speed)

7.564268359017664e-13
[2.7417807914068246e-15,-6.667344699708082e-16,-2.092798120185854e-15,4.4310493738516057e-14,-3.70727804254414e-14,1.4244184878175099e-15,1.533773746157506e-15,0.9999999999999813,4.016862682672356e-16]
2.7417807914068246e-15
2.1883587553041988e+16


### Regression model with one-hot encoding

In [41]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Split the data into training and testing sets
df_onehot_train, df_onehot_test = df_onehot.randomSplit([0.8, 0.2], seed=13)

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(df_onehot_train)

# Create predictions for the testing data
predictions = regression.transform(df_onehot_test)

# Calculate the RMSE on testing data
RegressionEvaluator(labelCol='duration').evaluate(predictions)

24/08/10 10:44:30 WARN Instrumentation: [714ff69b] regParam is zero, which might cause numerical instability and overfitting.


3.3211762982106225e-13

In [42]:
# Average speed in km per hour
avg_speed_hour = 60/regression.coefficients[0]
print(avg_speed_hour)

# Average minutes on ground at OGG
inter = regression.intercept
print(inter)

# Average minutes on ground at JFK
avg_ground_jfk = inter + regression.coefficients[3]
print(avg_ground_jfk)

# Average minutes on ground at LGA
avg_ground_lga = inter + regression.coefficients[4]
print(avg_ground_lga)

5803707886850709.0
-1.21874977156918e-12
-1.2539480943622785e-12
-1.172498320259883e-12


### Bucketing & One-hot encoding

In [43]:
from pyspark.ml.feature import Bucketizer, OneHotEncoder

# Create buckets at 3 hour intervals through the day
buckets = Bucketizer(splits=[0,3,6,9,12,15,18,21,24], inputCol='depart', outputCol='depart_bucket')

# Bucket the departure times
bucketed = buckets.transform(df_onehot)
display(bucketed.select('depart','depart_bucket').show(5))

# Create a one-hot encoder
onehot = OneHotEncoder(inputCols=['depart_bucket'], outputCols=['depart_dummy'])

# One-hot encode the bucketed departure times
df_onehot1 = onehot.fit(bucketed).transform(bucketed)
df_onehot1.select('depart', 'depart_bucket', 'depart_dummy').show(5)

+------+-------------+
|depart|depart_bucket|
+------+-------------+
|  8.18|          2.0|
|  7.17|          2.0|
| 21.17|          7.0|
| 12.92|          4.0|
| 13.33|          4.0|
+------+-------------+
only showing top 5 rows



None

+------+-------------+-------------+
|depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
|  8.18|          2.0|(7,[2],[1.0])|
|  7.17|          2.0|(7,[2],[1.0])|
| 21.17|          7.0|    (7,[],[])|
| 12.92|          4.0|(7,[4],[1.0])|
| 13.33|          4.0|(7,[4],[1.0])|
+------+-------------+-------------+
only showing top 5 rows



### Lasso & Ridge Regression

In [44]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
# Fit Lasso model (λ = 1, α = 1) to training data
regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(df_train)
# elasticNetParam: {0 for ridge & 1 for lasso}
# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(regression.transform(df_test))
print("The test RMSE is", rmse)
# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)
# Number of zero coefficients
zero_coeff = sum([beta==0 for beta in regression.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)

The test RMSE is 1.0214000829509382
[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9885520847824293,0.0]
Number of coefficients equal to 0: 8


## Pipeline

### Numerical Data

In [56]:
# Convert categorical strings to index values
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# One-hot encode index values
onehot = OneHotEncoder(
    inputCols=['org_idx', 'dow'],
    outputCols=['org_dummy', 'dow_dummy']
)

# Assemble predictors into a single column
assembler = VectorAssembler(inputCols=['km', 'org_dummy', 'dow_dummy'], outputCol='features')

# A linear regression object
regression = LinearRegression(labelCol='duration')

In [57]:
df_index.columns

['mon',
 'dom',
 'dow',
 'carrier',
 'org',
 'depart',
 'duration',
 'delay',
 'km',
 'label']

In [60]:
# Import class for creating a pipeline
from pyspark.ml import Pipeline

# Construct a pipeline
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])

# Split into training and testing sets in a 80:20 ratio
df_train, df_test = df_index.randomSplit([0.8, 0.2], seed=43)

# Train the pipeline on the training data
pipeline = pipeline.fit(df_train)

# # Make predictions on the testing data
# predictions = pipeline.transform(df_test)

24/08/10 10:51:06 WARN Instrumentation: [316883d8] regParam is zero, which might cause numerical instability and overfitting.


### Text Data

In [61]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression

# Break text into tokens at non-word characters
tokenizer = Tokenizer(inputCol='text', outputCol='words')

# Remove stop words
remover = StopWordsRemover(inputCol='words', outputCol='terms')

# Apply the hashing trick and transform to TF-IDF
hasher = HashingTF(inputCol="terms", outputCol="hash")
idf = IDF(inputCol="hash", outputCol="features")

# Create a logistic regression object and add everything to a pipeline
logistic = LogisticRegression()
pipeline = Pipeline(stages=[tokenizer, remover, hasher, idf, logistic])

### Cross validation

In [ ]:
# # Create an assembler object
# assembler = VectorAssembler(inputCols=[
#     'mon', 'depart', 'duration'
# ], outputCol='features')

# # Consolidate predictor columns
# df_index_assembler = assembler.transform(df_index.select('mon', 'dow', 'org', 'depart', 'duration', 'km', 'label'))

In [65]:
df_indexed.show(1)

+---+---+---+-------+---+------+--------+-----+-----+-----+-----------+-------+
|mon|dom|dow|carrier|org|depart|duration|delay|   km|label|carrier_idx|org_idx|
+---+---+---+-------+---+------+--------+-----+-----+-----+-----------+-------+
| 10| 10|  1|     OO|ORD|  8.18|      51|   27|253.0|    1|        2.0|    0.0|
+---+---+---+-------+---+------+--------+-----+-----+-----+-----------+-------+
only showing top 1 row



In [66]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create an empty parameter grid
params = ParamGridBuilder().build()

# Create objects for building and evaluating a regression model
regression = LinearRegression(labelCol='duration')
evaluator = RegressionEvaluator(labelCol='duration')

# Create a cross validator
cv = CrossValidator(estimator=regression, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'mon', 'depart', 'duration'
], outputCol='features')

# Consolidate predictor columns
df_assembler = assembler.transform(df_indexed)

df_train, df_test = df_assembler.randomSplit([0.8, 0.2], seed=43)
# Train and test model on multiple folds of the training data
cv = cv.fit(df_train)

# NOTE: Since cross-valdiation builds multiple models, the fit() method can take a little while to complete.

24/08/10 10:53:23 WARN Instrumentation: [a35e6320] regParam is zero, which might cause numerical instability and overfitting.
24/08/10 10:53:24 WARN Instrumentation: [de4ae38f] regParam is zero, which might cause numerical instability and overfitting.
24/08/10 10:53:25 WARN Instrumentation: [29a8335d] regParam is zero, which might cause numerical instability and overfitting.
24/08/10 10:53:26 WARN Instrumentation: [ad19039f] regParam is zero, which might cause numerical instability and overfitting.
24/08/10 10:53:26 WARN Instrumentation: [c589f20e] regParam is zero, which might cause numerical instability and overfitting.
24/08/10 10:53:27 WARN Instrumentation: [b13d3a94] regParam is zero, which might cause numerical instability and overfitting.


### pipeline with CV

In [73]:
# Create an indexer for the org field
indexer = StringIndexer(inputCol='org', outputCol='org_idx')

# Create an one-hot encoder for the indexed org field
onehot = OneHotEncoder(inputCols=['org_idx'], outputCols=['org_dummy'])

# Assemble the km and one-hot encoded fields
assembler = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features')

# Create a pipeline and cross-validator.
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=params,
                    evaluator=evaluator)

### Grid Search

In [74]:
# Create parameter grid
params = ParamGridBuilder()

# Add grids for two parameters
params = params.addGrid(regression.regParam, [0.01, 0.1, 1.0, 10.0]) \
               .addGrid(regression.elasticNetParam, [0.0, 0.5, 1.0])

# Build the parameter grid
params = params.build()
print('Number of models to be tested: ', len(params))

# Create cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

Number of models to be tested:  12


In [78]:
df_train, df_test = df_index.randomSplit([0.8, 0.2], seed=43)

# Model Fit
cv.fit(df_train, params=params)

# Get the best model from cross validation
best_model = cv.bestModel

# Look at the stages in the best model
print(best_model.stages)

# Get the parameters for the LinearRegression object in the best model
best_model.stages[3].extractParamMap()

# Generate predictions on testing data using the best model then calculate RMSE
predictions = best_model.transform(df_test)
print("RMSE =", evaluator.evaluate(predictions))

24/08/10 11:06:36 WARN CacheManager: Asked to cache already cached data.
24/08/10 11:06:36 WARN CacheManager: Asked to cache already cached data.


AttributeError: 'CrossValidator' object has no attribute 'bestModel'

### Ensemble

In [90]:
# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'mon', 'depart', 'duration'
], outputCol='features')

# Consolidate predictor columns
df_index_assembler = assembler.transform(df_index.select('mon', 'depart', 'duration', 'label'))

df_train, df_test = df_index_assembler.randomSplit([0.8, 0.2], seed=43)
df_train.show(2)

+---+------+--------+-----+----------------+
|mon|depart|duration|label|        features|
+---+------+--------+-----+----------------+
|  0|  5.75|      50|    0| [0.0,5.75,50.0]|
|  0|  5.75|     119|    0|[0.0,5.75,119.0]|
+---+------+--------+-----+----------------+
only showing top 2 rows



In [91]:
# Import the classes required
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
# Create model objects and train on training data
tree = DecisionTreeClassifier().fit(df_train)
gbt = GBTClassifier().fit(df_train)
# Compare AUC on testing data
evaluator = BinaryClassificationEvaluator()

print(evaluator.evaluate(tree.transform(df_test)))
print(evaluator.evaluate(gbt.transform(df_test)))
# Find the number of trees and the relative importance of features
print(gbt.trees)
print(gbt.featureImportances)

0.6401995200516241
0.6807341881901746
[DecisionTreeRegressionModel: uid=dtr_0c400ce796bd, depth=5, numNodes=63, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_dea0e8edbb12, depth=5, numNodes=63, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_7b211d22856c, depth=5, numNodes=63, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_bd5c564eecac, depth=5, numNodes=63, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_7f6caa740f5c, depth=5, numNodes=61, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_514fe69367c6, depth=5, numNodes=63, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_2f285047fe9d, depth=5, numNodes=63, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_96fa2f30cef9, depth=5, numNodes=63, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_f26a192786e7, depth=5, numNodes=63, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_fdb7958fa537, depth=5, numNodes=63, numFeatures=3, DecisionTreeRegressionModel: uid=dtr_0f6944fc0503, depth=5, numNodes=61, numFeatu

In [93]:
from pyspark.ml.classification import RandomForestClassifier
# Create a random forest classifier
forest = RandomForestClassifier()
# Create a parameter grid
params = ParamGridBuilder() \
            .addGrid(forest.featureSubsetStrategy, ['all', 'onethird', 'sqrt', 'log2']) \
            .addGrid(forest.maxDepth, [2, 5, 10]) \
            .build()
# Create a binary classification evaluator
evaluator = BinaryClassificationEvaluator()
# Create a cross-validator
cv = CrossValidator(estimator=forest, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

In [ ]:
# Create a random forest classifier
forest = RandomForestClassifier()
# Create a parameter grid
params = ParamGridBuilder() \
            .addGrid(forest.featureSubsetStrategy, ['all', 'onethird', 'sqrt', 'log2']) \
            .addGrid(forest.maxDepth, [2, 5, 10]) \
            .build()
# Create a binary classification evaluator
evaluator = BinaryClassificationEvaluator()
# Create a cross-validator
cv = CrossValidator(estimator=forest, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

In [94]:
# Average AUC for each parameter combination in grid
print(cv.avgMetrics)
# Average AUC for the best model
print(max(cv.avgMetrics))  # 0.68
# What's the optimal parameter value for maxDepth?
print(cv.bestModel.explainParam('maxDepth'))
# What's the optimal parameter value for featureSubsetStrategy?
print(cv.bestModel.explainParam('featureSubsetStrategy'))
# AUC for best model on testing data
print(evaluator.evaluate(cv.bestModel.transform(flights_test)))

AttributeError: 'CrossValidator' object has no attribute 'avgMetrics'